In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import datetime
from collections import defaultdict

In [ ]:
from utils import load_data

FOLDER_PATH = "sentiment_datasets/projekt2_data"

data_df = load_data(FOLDER_PATH, df_delimiter=",")
data_df.head(10)

In [ ]:
data_df["Phrase"].str.split().str.len().plot(kind="hist", title="number of tokens in line distribution", grid=True, figsize=(10,10), bins=50);

# ELMO

In [ ]:
from preprocess_data import preprocess_data_word
from layers import ElmoLayer
from utils import get_proper_callback
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
# from livelossplot import PlotLossesKeras
from utils_result import save_report

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test), data = preprocess_data_word(data_df, x_label="Phrase", y_label="Sentiment", max_len_seq=20)

In [ ]:
callback_params = {
    "histogram_freq": 1,
    "write_graph": True,
    "write_images": True
}

fit_params = {
    "x": x_train,
    "y": y_train,
    "batch_size": 64,
    "epochs": 50,
    "validation_data": (x_val, y_val)    
}

## Word Level

### RNN

In [ ]:
from models import model_rnn

In [ ]:
params_elmo_tokens_rnn = {
    "shape": (1, ),
    "input_dtype": x_train.dtype,
    "emb_layer": ElmoLayer(trainable=True, dict_output="elmo"),
    "out_units": np.unique(y_train).shape[0],
    "out_activation": "softmax",
    "loss": "sparse_categorical_crossentropy",
    "optimizer": "rmsprop",
    "is_bert": False
}

model_elmo_rnn = model_rnn(params_elmo_tokens_rnn)

In [ ]:
# fit_params["callbacks"] = [get_proper_callback(PlotLossesKeras)]
fit_params["callbacks"] = [EarlyStopping(patience=5)]

history_rnn = model_elmo_rnn.fit(**fit_params)
save_report(model_elmo_rnn, "model_elmo_rnn", history_rnn.history)

### CNN

In [ ]:
from models import model_cnn

In [ ]:
params_elmo_tokens_cnn = {
    "shape": (1, ),
    "input_dtype": x_train.dtype,
    "emb_layer": ElmoLayer(trainable=True, dict_output="elmo"),
    "out_units": np.unique(y_train).shape[0],
    "out_activation": "softmax",
    "loss": "sparse_categorical_crossentropy",
    "optimizer": "rmsprop",
    "is_bert": False
}

model_elmo_cnn = model_cnn(params_elmo_tokens_cnn)

In [ ]:
# fit_params["callbacks"] = [get_proper_callback(PlotLossesKeras)]
fit_params["callbacks"] = [EarlyStopping(patience=5)]

history_cnn = model_elmo_cnn.fit(**fit_params)
save_report(model_elmo_cnn, "model_elmo_cnn", history_cnn.history)

## Phrase Level FFNN

In [ ]:
from models import model_ffnn

In [ ]:
params_elmo_raw = {
    "shape": (1, ),
    "input_dtype": x_train.dtype,
    "emb_layer": ElmoLayer(trainable=True, dict_output="default"),
    "out_units": np.unique(y_train).shape[0],
    "out_activation": "softmax",
    "loss": "sparse_categorical_crossentropy",
    "optimizer": "rmsprop",
    "is_bert": False
}

model_elmo_ffnn = model_ffnn(params_elmo_raw)

In [ ]:
# fit_params["callbacks"] = [get_proper_callback(PlotLossesKeras)]
fit_params["callbacks"] = [EarlyStopping(patience=5)]

history_ffnn = model_elmo_ffnn.fit(**fit_params)
save_report(model_elmo_ffnn, "model_elmo_ffnn", history_ffnn.history)